<a href="https://colab.research.google.com/github/doukansurel/Retrieval-Augmented-Generation/blob/main/HyDEQueryTransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index pypdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

#Drive Bağlanma

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

#Kurulum

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.indices.query.query_transform import HyDEQueryTransform
from llama_index.query_engine.transform_query_engine import (
    TransformQueryEngine,
)
from IPython.display import Markdown, display

#Belge Yükleme

In [5]:
documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/RAG/Belgeler/satis-giderleri-tarifesi-4-1.pdf"]).load_data()

#İndeksleme

In [6]:
index = VectorStoreIndex.from_documents(documents)

In [9]:
query_str = "Sicile kayıtlı motorlu kara araçlarına ilişkin satış gideri ile taşınır mallara ilişkin satış giderinin farkı nedir türkçe yaz?"

In [10]:
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>Sicile kayıtlı motorlu kara araçlarına ilişkin satış gideri, aracın muhafazası, kıymet takdiri ve satış giderleri esas alınarak belirlenirken, taşınır mallara ilişkin satış gideri ise kıymet takdiri ve satış giderleri esas alınarak belirlenmektedir.</b>

# HyDE Query Transform Kullanarak

In [11]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>Sicile kayıtlı motorlu kara araçlarına ilişkin satış gideri, aracın muhafazası, kıymet takdiri ve satış giderleri esas alınarak belirlenirken, taşınır mallara ilişkin satış gideri ise kıymet takdiri ve satış giderleri esas alınarak belirlenmektedir.</b>

In [12]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [13]:
hyde_doc

'Sicile kayıtlı motorlu kara araçlarına ilişkin satış gideri ile taşınır mallara ilişkin satış gideri arasında bazı önemli farklar bulunmaktadır. İlk olarak, Sicile kayıtlı motorlu kara araçlarına ilişkin satış gideri, özellikle otomobil, motosiklet veya kamyon gibi taşıtların satışıyla ilgili olan masrafları ifade eder. Bu masraflar, aracın reklamı, pazarlaması, sigortası, bakımı ve vergileri gibi çeşitli unsurları içerebilir. Ayrıca, aracın satış fiyatının belirlenmesi ve müşteriye sunulması için yapılan harcamalar da bu gider kategorisine dahil edilir.\n\nÖte yandan, taşınır mallara ilişkin satış gideri, genel olarak herhangi bir türdeki malın satışıyla ilgili olan masrafları ifade eder. Bu masraflar, malın depolanması, paketlenmesi, nakliyesi ve dağıtımı gibi işlemleri içerebilir. Ayrıca, malın satış fiyatının belirlenmesi, etiketlenmesi ve müşteriye sunulması için yapılan harcamalar da bu gider kategorisine dahil edilir.\n\nBu iki gider kategorisi arasındaki en önemli fark, hangi 

**Sorgu bağlam olmadan yanlış yorumlanabildiğinde HyDE yanıltıcı olabilir.**

In [17]:
query_str = "Satış giderleri nedir?"

In [18]:
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>Satış giderleri, bir satış işlemi gerçekleştirildiğinde ödenmesi gereken ücretlerdir. Bu ücretler, satışın yapıldığı malın niteliğine ve değerine göre belirlenir. Satış giderleri tablosunda yer alan ücretler esas alınarak hesaplanır.</b>

In [19]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>Satış giderleri, bir satış işlemi gerçekleştirildiğinde ödenmesi gereken ücretlerdir. Bu ücretler, satışın yapıldığı malın niteliğine ve değerine göre belirlenir. Satış giderleri tablosunda yer alan ücretler esas alınarak hesaplanır.</b>

In [20]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [21]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [22]:
hyde_doc

'Satış giderleri, bir şirketin ürün veya hizmetlerini pazarlamak ve satmak için yaptığı harcamalardır. Bu harcamalar, genellikle reklam ve pazarlama faaliyetleri, satış personeli maaşları, satış ofislerinin kirası, satış ekipmanı ve araçları gibi çeşitli kalemleri içerir. Şirketler, ürünlerini veya hizmetlerini müşterilere tanıtmak, satışları artırmak ve kar elde etmek için satış giderlerine yatırım yaparlar. Reklam ve pazarlama faaliyetleri, televizyon, radyo, gazete, dergi, internet ve sosyal medya gibi çeşitli kanallar aracılığıyla gerçekleştirilebilir. Satış personeli maaşları, satış ekibinin üyelerine ödenen ücretleri içerir ve genellikle satış performansına bağlı olarak değişir. Satış ofislerinin kirası, şirketin satış faaliyetlerini yürütmek için kullandığı fiziksel mekanların maliyetini temsil eder. Satış ekipmanı ve araçları, satış personelinin işlerini daha etkili bir şekilde yapmalarını sağlamak için kullanılan araçları ve teknolojik ekipmanları içerir. Satış giderleri, şirk

**HyDE açık uçlu sorgulara önyargılı davranabilir**

In [ ]:
query_str="Sicile kayıtlı motorlu kara araçlarına ilişkin satış giderleri ile taşınmaz mallara, taşınır mallara ve diğer mallara ilişkin satış giderleri arasındaki farklılıkları açıklar mısınız?"

In [23]:
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>Satış giderleri, bir satış işlemi gerçekleştirildiğinde ödenmesi gereken ücretlerdir. Bu ücretler, satış talep edilen malın niteliğine ve türüne göre belirlenir. Örneğin, taşınmaz mallar için satış giderleri, kıymet takdiri ve satış giderleri esas alınarak belirlenirken, sicile kayıtlı motorlu kara araçları için farklı bir ücret tablosu kullanılır. Diğer mallar için ise satış giderleri tablosunda belirtilen ücretler esas alınır. Satış giderleri, satış talep edilen malın satışı için önceden belirlenen bir tarifeye göre hesaplanır ve satış işlemi gerçekleştirilmeden önce avans olarak ödenmesi gerekmektedir.</b>

In [24]:
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>Satış giderleri, bir satış işlemi gerçekleştirildiğinde ödenmesi gereken ücretlerdir. Bu ücretler, satış talep edilen malın niteliğine ve değerine göre belirlenir. Satış giderleri tablosunda yer alan ücretler esas alınarak hesaplanır.</b>